<a id='report-scopes-earnings-kpi'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Reports for Earnings KPIs</span> </p>

Earnings KPIs are defined at ["Report Scopes Earnings KPIs"](../../ifrs17/Report/ReportScopesEarningsKpis) 

# Import Modules
We need to import the Calculation Engine and Initialize the system with with reference data.

In [0]:
#!import "../Initialization/InitSystemorphToMemory"

In [0]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.InitializeFrom(DataSource);

ifrs17.Reset(Workspace)

## Stuff I had to add

In [0]:
using Systemorph.Vertex.Pivot.Builder;
using Systemorph.Vertex.Pivot.Reporting.Builder;
using System.Collections.Immutable;

public static DataCubeReportBuilder<IDataCube<TVariable>, TVariable, TVariable ,TVariable> ReportGridOptions<TVariable>(
    this DataCubePivotBuilder<IDataCube<TVariable>, TVariable, TVariable, TVariable> pivotBuilder,
    int reportHeight = 700,
    int valueColumnWidth = 250,
    int headerColumnWidth = 250,
    int groupDefaultExpanded = 2)
    => pivotBuilder.ToTable().WithOptions(go => go
                                     .WithColumns(cols => cols.Modify("Value",c => c.WithWidth(valueColumnWidth).WithFormat("typeof(value) == 'number' ? new Intl.NumberFormat('en',{ minimumFractionDigits:3, maximumFractionDigits:3 }).format(value) : value")))
                                     .WithRows(rows => rows
                                               .Where(r => !(r.RowGroup.Coordinates.Last() == "NullGroup"))
                                               .Select(r => r with { RowGroup = r.RowGroup with { Coordinates = r.RowGroup.Coordinates.Where(c => c != "NullGroup").ToImmutableList() } })
                                               .ToArray())
                                     .HideRowValuesForDimension("Novelty")
                                     .HideRowValuesForDimension("Projection")
                                     .WithAutoGroupColumn(c => c.WithWidth(headerColumnWidth) with { Pinned = "left" })
                                     with { Height = reportHeight, GroupDefaultExpanded = groupDefaultExpanded, OnGridReady = null } );

In [0]:
using Systemorph.Vertex.Pivot.Builder.Interfaces;
using Systemorph.Vertex.DataSource;
public static DataCubePivotBuilder<IDataCube<TElement>, TElement, TElement, TElement> ForSlicedDataCube<TElement>(this IPivotFactory factory, 
                                            IDataCube<TElement> cube,
                                            IDataSourceVariable dataSource,
                                            List<string> rowsSlice = null, 
                                            List<string> columnsSlice = null)
{
    rowsSlice ??= new List<string>() {"Novelty", "VariableType"};
    columnsSlice ??= new List<string>() {"Currency", "GroupOfContract", "EconomicBasis"};
    return factory.ForDataCube(cube)
                .WithQuerySource(dataSource)
                .SliceRowsBy(rowsSlice.ToArray())
                .SliceColumnsBy(columnsSlice.ToArray());
}

In [0]:
// public static IDataCube<ReportVariable> DivideKpi  (this IDataCube<ReportVariable> a, IDataCube<ReportVariable> b) {
//     var denominator = b.AggregateBy("Currency", "Scenario", "Projection", "Portfolio");
//     var numerator = a.AggregateBy("Currency", "Scenario", "Projection", "Portfolio");

//     return numerator.Select(x => new { Order = 1, Position = x })
//     .Concat(denominator.Select(x => new { Order = 2, Position = x }))
//     .OrderBy(x => x.Order)
//     .Select(x => x.Position)
//     .GroupBy(x => new {x.Currency, x.Scenario, x.Projection, x.Portfolio})
//     .SelectToDataCube(x => new ReportVariable(x.Last()) {
//         Value = Math.Abs(x.Last().Value) < Precision
//             ? -999
//             : (x.ElementAtOrDefault(0)?.Value ?? 0) / x.Last().Value
//         });
// }


In [0]:
// public static IDataCube<ReportVariable> DivideKpi  (this IDataCube<ReportVariable> a, IDataCube<ReportVariable> b) {
//     return a.Select(x => new { Type = "Numerator", Position = x })
//         .Concat(b.Select(x => new { Type = "Denominator", Position = x }))
//         .GroupBy(x => new {x.Position.Currency, x.Position.Scenario, x.Position.Projection, x.Position.Portfolio})
//         .SelectToDataCube(x => {
//             var d = x.SingleOrDefault(y => y.Type == "Denominator")?.Position ?? null;
//             return (d == null || Math.Abs(d.Value) < Precision)
//             ? null
//                 : new ReportVariable(d) { Value = 
//                     (x.SingleOrDefault(y => y.Type == "Numerator")?.Position.Value ?? 0) / d.Value 
//                 };
//             }
//         )
//         .Where(x => x != null)
//         .ToDataCube();
// }

In [0]:
public static IDataCube<ReportVariable> DivideKpi  (this IDataCube<ReportVariable> a, IDataCube<ReportVariable> b) {
    return a.Select(x => new { Type = "Numerator", Position = x })
        .Concat(b.Select(x => new { Type = "Denominator", Position = x }))
        .GroupBy(x => new {x.Position.Currency, x.Position.Scenario, x.Position.Projection, x.Position.Portfolio})
        .Select(x => {
            var d = x.Where(y => y.Type == "Denominator").Aggregate()?.Position;
            return (d == null || Math.Abs(d.Value) < Precision)
            ? null
                : new ReportVariable(d) { Value = (x.Where(y => y.Type == "Numerator").Aggregate().Position.Value)
                    //(x.Where(y => y.Type == "Numerator").Aggregate().Position.Value) / d.Value 
                };
            }
        )
        .Where(x => x != null)
        .ToDataCube();
}

In [0]:
public interface Kpi: IScope<(ReportIdentity Id, CurrencyType CurrencyType), ReportStorage>, IDataCube<ReportVariable> {
    private FinancialPerformanceAlternative FinancialPerformanceAlternative => GetScope<FinancialPerformanceAlternative>(Identity);
    IDataCube<ReportVariable> Profit => FinancialPerformanceAlternative.FinancialPerformanceAlternative.AggregateBy("Currency", "Scenario", "Projection", "Portfolio");
    IDataCube<ReportVariable> InsuranceRevenue => FinancialPerformanceAlternative.InsuranceRevenue.AggregateBy("Currency", "Scenario", "Projection", "Portfolio");
    
    IDataCube<ReportVariable> Kpi => Profit.DivideKpi(InsuranceRevenue);
}

In [0]:
reportIdentities.Select(x=> x.Item1).AggregateBy("Scenario", "Projection", "IsReinsurance", "IsOci").Select(x => (x, CurrencyType.Contractual))

In [0]:
var scopeKpi = universe.GetScopes<Kpi>(reportIdentities.Select(x=> x.Item1).AggregateBy("Scenario", "Projection", "IsReinsurance", "IsOci").Select(x => (x, CurrencyType.Contractual))).Aggregate();

In [0]:
var p = scopeKpi.Profit.AggregateBy("Currency", "Scenario", "Projection", "Portfolio").ToDataCube();
var d = scopeKpi.InsuranceRevenue.AggregateBy("Currency", "Scenario", "Projection", "Portfolio").ToDataCube();
var k = scopeKpi.Kpi.AggregateBy("Currency", "Scenario", "Projection", "Portfolio").ToDataCube();

In [0]:
p

In [0]:
scopeKpi.Profit

In [0]:
p

In [0]:
universe.GetScopes<Kpi>(reportIdentities)//.Select(x => x.Profit)//.SelectMany(x => x.Kpi)

In [0]:
p.Select(x => new { Type = "Numerator", Position = x })
    .Concat(d.Select(x => new { Type = "Denominator", Position = x }))
    .GroupBy(x => new {x.Position.Currency, x.Position.Scenario, x.Position.Projection, x.Position.Portfolio})
    .SelectToDataCube(x => {
        var d = x.Where(y => y.Type == "Denominator").Aggregate()?.Position;
        return (d == null || Math.Abs(d.Value) < Precision)
            ? null
            : new {den = d.Value, num = (x.Where(y => y.Type == "Numerator").Aggregate()?.Position.Value), ratio = ((x.Where(y => y.Type == "Numerator").Aggregate()?.Position.Value)) / d.Value };
            //ReportVariable(d) { Value = (x.SingleOrDefault(y => y.Type == "Numerator")?.Position.Value ?? 0) / d.Value };
        }
    ).Where(x => x != null)
    //.Select(x => new {count = x.Count(), num = x.First().Value, den = x.Last().Value, element1 = x.ElementAtOrDefault(0)?.Value ?? 0, ratio = (x.ElementAtOrDefault(0)?.Value ?? 0) / x.Last().Value})

# Report Setup

## Report Framework and Scope (args)

In [0]:
var reportStorage = new ReportStorage(Workspace, Report, Export);
await reportStorage.InitializeReportIndependentCacheAsync();
var universe = Scopes.ForSingleton().WithStorage(reportStorage).ToScope<IUniverse>();

In [0]:
((int Year, int Month) Period, string ReportingNode, string Scenario, CurrencyType CurrencyType) currentArgs = ((2021, 3), "G", null, CurrencyType.Contractual);
await reportStorage.InitializeAsync(currentArgs.Period, currentArgs.ReportingNode, currentArgs.Scenario, currentArgs.CurrencyType);
var reportIdentities = reportStorage.GetIdentities(currentArgs.Period, currentArgs.ReportingNode, currentArgs.Scenario, currentArgs.CurrencyType);

In [0]:
 reportStorage.GetIdentities(currentArgs.Period, currentArgs.ReportingNode, currentArgs.Scenario, currentArgs.CurrencyType)

## Report Settings

You can set filters to any property of a [Report Variable](https://portal.systemorph.cloud/project/ifrs17/env/dev/DataModel/DataStructure#report-variable) data cube. I added Portfolio Filters as an example below (as it was convenient for analysis). This filter will be applicable to all reports below. Individual filters for each report can also be applied directly below.

In [0]:
Func<ReportVariable, bool> portfolioFilters =
    x => true;
    //x => x.Portfolio == "G.BBA";
    //x => x.Portfolio == "R.BBA";
    //x => x.Portfolio == "G.BBA" || x.Portfolio == "R.BBA" || x.Portfolio == "R.PAA";
    //x => x.Portfolio == "R.PAA";

# Reports
You can comment/uncomment the cells below as appropriate for your analysis.

In [0]:
var kpi = universe.GetScopes<Kpi>(reportIdentities)//.Aggregate()
                  //.Filter(portfolioFilters)
                  //.Filter(x => Math.Abs(x.Value) > Precision)
                  ;

var kpiRowSlices = new List<string> {"THIS"};
var kpiColumnSlices = new List<string> {"Currency", "Scenario", "Projection", "Portfolio"};


 await 
 //Report.ForSlicedDataCube(kpi, DataSource, kpiRowSlices, kpiColumnSlices)
 Report.ForDataCubes(kpi)
      .SliceColumnsBy("Currency", "Portfolio")
      .SliceRowsBy("THIS")
      .ReportGridOptions()
      .ExecuteAsync()

In [0]:
90.470/-55.035


In [0]:
var fpalt = universe.GetScopes<FinancialPerformanceAlternative>(reportIdentities).Aggregate().FinancialPerformanceAlternative.ToDataCube()
    .Filter(portfolioFilters)
    ;
            
var fpaltRowSlices = new List<string>() {"VariableType", "EstimateType"};
//var fpaltColumnSlices = new List<string>() {"Currency", "LiabilityType","GroupOfContract"};
var fpaltColumnSlices = new List<string>() {"Currency", "Scenario", "Projection", "Portfolio"};
await Report.ForSlicedDataCube(fpalt, DataSource, fpaltRowSlices, fpaltColumnSlices)
    .ReportGridOptions(headerColumnWidth: 500, groupDefaultExpanded: 3)
    .ExecuteAsync()

## Best Estimate PV
This reports shows the Best Estimate Present Value of Future Cash Flows (excluding Risk Adjustment) and can be used to check that the input files was correctly imported into the tool.

In [0]:
var pvs = //(universe.GetScopes<LockedBestEstimate>(projectionIdentities).Aggregate().LockedBestEstimate + universe.GetScopes<CurrentBestEstimate>(projectionIdentities).Aggregate().CurrentBestEstimate).Filter(("EconomicBasis", "L"))
            universe.GetScopes<BestEstimate>(reportIdentities).Aggregate().BestEstimate
                  .Filter(portfolioFilters)
                  .Filter(x => Math.Abs(x.Value) > Precision)
                  ;

var pvRowSlices = new List<string> {"Novelty", "VariableType"};
var pvColumnSlices = new List<string> {"Currency", "LiabilityType", "EconomicBasis", "GroupOfContract", "AccidentYear", "AmountType"};
await Report.ForSlicedDataCube(pvs, DataSource, pvRowSlices, pvColumnSlices)
    .ReportGridOptions()
    .ExecuteAsync()

## Actuals vs Expected
This report shows the delta between the expected cash flow (nominal expected release) and the actuals. The initial assumption is that they are the same (i.e. no Experience Variance) and this report can be used to check the numbers.

In [0]:
var aeas = universe.GetScopes<ExperienceAdjustment>(projectionIdentities).Aggregate().ActuarialExperienceAdjustment
    .Filter(x => !(x.LiabilityType == LiabilityTypes.LRC && reportStorage.GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.CL)))
    .Filter(x => !(x.LiabilityType == LiabilityTypes.LRC && x.ValuationApproach == ValuationApproaches.PAA && (reportStorage.GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.AC) || reportStorage.GetHierarchy<AmountType>().Ancestors(x.AmountType, includeSelf: true).Any(x => x.SystemName == AmountTypes.AE))))
    .Filter(portfolioFilters)
    ;

var aeasRowSlices = new List<string> {"AmountType", "EstimateType"};
var aeasColumnSlices = new List<string> {"Currency", "Projection", "GroupOfContract"};

await Report.ForSlicedDataCube(aeas, DataSource, aeasRowSlices, aeasColumnSlices)
    .ReportGridOptions(groupDefaultExpanded: 0)
    .ExecuteAsync()

## Deferrals
This report show the amortization of the Insurance Acquisition Costs (deferrals).

In [0]:
var deferrals = universe.GetScopes<Deferrals>(projectionIdentities).Aggregate().Deferrals
    .Filter(portfolioFilters)
    ;
                
var deferralsRowSlices = new List<string> {"Novelty", "VariableType"};
var deferralsColumnSlices = new List<string> {"Currency", "Projection", "LiabilityType", "GroupOfContract"};

await Report.ForSlicedDataCube(deferrals, DataSource, deferralsRowSlices, deferralsColumnSlices)
    .ReportGridOptions()
    .ExecuteAsync()

## CSM/LC/LoReCo
This report show the amortization of the Contractual Service Margin (CSM), Loss Component (LC) and Loss Recovery Component (LoReCo). The LC only applies to Gross business, while the LoReCo only applies to Reinsurance held. Furthermore, this report is only applicable for BBA, as these quantities are not defined for PAA.

In [0]:
var csmLcLoreco = (universe.GetScopes<Csm>(projectionIdentities).Aggregate().Csm + universe.GetScopes<Lc>(projectionIdentities).Aggregate().Lc + universe.GetScopes<Loreco>(projectionIdentities).Aggregate().Loreco)
    .Filter(portfolioFilters)
    ;
                        
var csmLcLorecoRowSlices = new List<string> {"Novelty","VariableType"};
var csmLcLorecoColumnSlices = new List<string> {"Currency", "Projection","GroupOfContract", "EstimateType"};
await Report.ForSlicedDataCube(csmLcLoreco, DataSource, csmLcLorecoRowSlices, csmLcLorecoColumnSlices)
    .ReportGridOptions()
    .ExecuteAsync()

## IFRS17 Actuarial Balance Sheet
This is the IFRS 17 Liability Actuarial Balance Sheet. The report shows the individual contributions of the LRC and LIC liabilities to the Balance Sheet.

In [0]:
var abs = (universe.GetScopes<LrcActuarial>(projectionIdentities).Aggregate().LrcActuarial + universe.GetScopes<LicActuarial>(projectionIdentities).Aggregate().LicActuarial)
    .Filter(portfolioFilters)
    ;

var absRowSlices = new List<string> {"Novelty","VariableType"};
var absColumnSlices = new List<string> {"Currency", "Projection", "LiabilityType", "GroupOfContract", "EstimateType" };
//var absColumnSlices = new List<string> {"Currency", "AnnualCohort", "GroupOfContract", "EstimateType" };
await Report.ForSlicedDataCube(abs, DataSource, absRowSlices, absColumnSlices)
    .ReportGridOptions()
    .ExecuteAsync()

## IFRS17 Financial Performance
This is the IFRS 17 Financial Performance, which include section for the Total Comprehensive Income, Profit and Loss and Other Comprehensive Income. As prescribe by the standard, the Profit and Loss is further broken down into Insurance Revenue, Insurance Service Expense and Insurance Finance Income/Expense.

In [0]:
var fpalt = universe.GetScopes<FinancialPerformanceAlternative>(reportIdentities).Aggregate().FinancialPerformanceAlternative.ToDataCube()
    .Filter(portfolioFilters)
    ;
            
var fpaltRowSlices = new List<string>() {"VariableType", "EstimateType"};
//var fpaltColumnSlices = new List<string>() {"Currency", "LiabilityType","GroupOfContract"};
var fpaltColumnSlices = new List<string>() {"Currency", "Scenario", "Projection", "Portfolio"};
await Report.ForSlicedDataCube(fpalt, DataSource, fpaltRowSlices, fpaltColumnSlices)
    .ReportGridOptions(headerColumnWidth: 500, groupDefaultExpanded: 3)
    .ExecuteAsync()